In [1]:
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrcra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [3]:
id = "051a2523-b8c4-4958-b166-c2fa8fb89059"
id = "1b22edff-bec9-40e8-8c8b-515655dd0377"

In [4]:
print("Getting number of characters in text field...")
sql = f"""
SELECT element_text
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER') 
AND id = '{id}';
"""
print(sql)

Getting number of characters in text field...

SELECT element_text
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER') 
AND id = '1b22edff-bec9-40e8-8c8b-515655dd0377';



In [5]:
cur.execute(sql)
text = cur.fetchone()[0]

num_chars = len(text)
print(text)
print(f"Num in text field {id[:8]}... is {num_chars:,}")

Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.
Num in text field 1b22edff... is 373


In [6]:
from summarizer import Summarizer

# Create a BERT extractive summarizer
summarizer = Summarizer()

# Generate the summary
summary = summarizer(
    text, min_length=100, max_length=200
)  # You can adjust the min_length and max_length parameters


print("Original Text:")
print(text)
print("\nSummary:")
print(summary)

Original Text:
Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.

Summary:
Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.


In [7]:
sql = f"""UPDATE tbl_doc_elements SET summary = '{summary}' WHERE id = '{id}';
"""
print(sql)

UPDATE tbl_doc_elements SET summary = 'Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.' WHERE id = '1b22edff-bec9-40e8-8c8b-515655dd0377';



In [8]:
cur.execute(sql)
conn.commit()

In [9]:
cur.close()
conn.close()

In [10]:
# cur.close()
# conn.close()